In [7]:
!pip install language-tool-python


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import heapq
import language_tool_python
import ipywidgets as widgets
from IPython.display import display, clear_output

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Tokenize the text into words
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)

    # Remove stopwords and stem the words
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    words = [ps.stem(word.lower()) for word in words if word.lower() not in stop_words]

    return words

def calculate_word_frequencies(words):
    # Calculate word frequencies
    word_freq = FreqDist(words)

    return word_freq

def calculate_sentence_scores(sentences, word_freq):
    sentence_scores = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence.lower()):
            if word in word_freq.keys():
                if len(sentence.split(' ')) < 30:
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_freq[word]
                    else:
                        sentence_scores[sentence] += word_freq[word]

    return sentence_scores

def generate_summary(text, num_sentences=3):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Preprocess the text
    words = preprocess_text(text)

    # Calculate word frequencies
    word_freq = calculate_word_frequencies(words)

    # Calculate sentence scores
    sentence_scores = calculate_sentence_scores(sentences, word_freq)

    # Select top 'num_sentences' sentences with highest scores
    summary_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)

    # Generate the summary
    summary = ' '.join(summary_sentences)

    return summary

def check_grammar(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return matches

def on_summarize_button_clicked(b):
    clear_output()
    user_text = text_input.value
    summary = generate_summary(user_text)
    display(summary_output)
    summary_output.value = summary

def on_grammar_check_button_clicked(b):
    clear_output()
    user_text = text_input.value
    matches = check_grammar(user_text)
    if len(matches) == 0:
        display(widgets.HTML("No grammar issues found."))
    else:
        display(widgets.HTML("<b>Grammar issues found:</b>"))
        for match in matches:
            display(widgets.HTML(f"<p>{match}</p>"))

# Text input widget
text_input = widgets.Textarea(
    value='',
    placeholder='Enter text here...',
    description='Text:',
    disabled=False
)

# Summarize button
summarize_button = widgets.Button(
    description='Summarize',
    button_style='success'
)
summarize_button.on_click(on_summarize_button_clicked)

# Grammar check button
grammar_check_button = widgets.Button(
    description='Grammar Check',
    button_style='info'
)
grammar_check_button.on_click(on_grammar_check_button_clicked)

# Output widget for summary
summary_output = widgets.HTML()

# Display widgets
display(text_input)
display(summarize_button)
display(grammar_check_button)


HTML(value='')